In [2]:
from io import BytesIO
import json
import time

from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage,
    ImageMessage,
    FlexMessage,
    FlexContainer,
    QuickReply,
    QuickReplyItem,
    MessageAction
)
from linebot.v3.webhooks import (
    MessageEvent,
    PostbackEvent,
    TextMessageContent,
    ImageMessageContent,
    LocationMessageContent
)
import cwa
from PIL import Image
from google.cloud import storage


with open('env.json', encoding='utf-8') as f:
    env = json.load(f)


storage_client = storage.Client.from_service_account_json('service.json')
bucket = storage_client.bucket('myfirstprojectnumberonebucket')

app = Flask(__name__)

configuration = Configuration(access_token=env.get('CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(env.get('CHANNEL_SECRET'))


@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    print(event.message.text)  #
    print(event.source.user_id) #
    print(event.timestamp)
    print(event.reply_token)

    messages = []
    
    ask = event.message.text
    ask_map = {'hello': '我很好', 'hi': '您哪位'}
    ans = ask_map.get(ask)
    if not ans:
        if ask == 'me':  # me handling
            with open('carousel.json', encoding='utf-8') as f:
                flex_dict = json.load(f)
            flex_message = FlexMessage(altText='me', contents=FlexContainer.from_dict(flex_dict))
            messages.append(flex_message)
        elif ask == 'action':  # action handling
            with open('action.json', encoding='utf-8') as f:
                flex_dict = json.load(f)
            flex_message = FlexMessage(altText='action', contents=FlexContainer.from_dict(flex_dict))
            messages.append(flex_message)
        elif ask in '12345':
            n = int(ask)
            with open('me.json', encoding='utf-8') as f:
                me_dict = json.load(f)
            car = {"type": "carousel", "contents": [me_dict]*n}
            flex_message = FlexMessage(altText=f'{n} me', contents=FlexContainer.from_dict(car))
            messages.append(flex_message)
        else:  # weather
            ans = cwa.cwa2(ask, env.get('CWA_KEY'))
            ans = cwa.tostr(ans, '\n') or '無此站'

            item1 = QuickReplyItem(action=MessageAction(label='文化大學', text='文化大學'))
            item2 = QuickReplyItem(action=MessageAction(label='陽明山', text='陽明山'))
            items = [item1, item2]
            quick_reply = QuickReply(items=items)

            messages.append(TextMessage(text=ans, quick_reply=quick_reply))
    else:  # hello
        messages.append(TextMessage(text=ans))
    
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=messages
            )
        )

@handler.add(MessageEvent, message=LocationMessageContent)
def handle_message(event):
    print(event.source.user_id) #
    print(event.timestamp)
    print(event.reply_token)

    print(event.message.latitude)
    print(event.message.longitude)
    site = (event.message.latitude, event.message.longitude)
    ans = cwa.cwa2(site, env.get('CWA_KEY'))
    ans = cwa.tostr(ans, '\n') or '無此站'
    
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=ans)]
            )
        )

@handler.add(MessageEvent, message=ImageMessageContent)
def handle_content_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_blob_api = MessagingApiBlob(api_client)
        message_content = line_bot_blob_api.get_message_content(message_id=event.message.id)
        image = Image.open(BytesIO(message_content))
        text = f'{image.height} X {image.width}'

        # upload image
        blob_name = f'{event.source.user_id}_{event.message.id}'
        blob = bucket.blob(blob_name)
        blob.upload_from_string(message_content, content_type='image/jpeg')
        url = blob.public_url
        url = blob.generate_signed_url(int(time.time())+60)
        image_message = ImageMessage(original_content_url=url,
                                     preview_image_url=url)
        
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[
                    TextMessage(text=text),
                    image_message
                ]
            )
        )


@handler.add(PostbackEvent)
def handle_message(event):
    print(event.source.user_id) #
    print(event.timestamp)
    print(event.reply_token)
    print(event.postback.data)
    
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=f'i see {event.postback.data}')]
            )
        )

if __name__ == "__main__":
    app.run(port=8080)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8080
Press CTRL+C to quit


大安森林
U091e03d020e37cce02ba9fd2f03377ec
1763196636029
b14f76bc49bb4e9b829404ca4771548c


127.0.0.1 - - [15/Nov/2025 16:50:38] "POST /callback HTTP/1.1" 200 -


文化大學
U091e03d020e37cce02ba9fd2f03377ec
1763196653951
83e63116abd54353b8cbb790ef5af130


127.0.0.1 - - [15/Nov/2025 16:50:56] "POST /callback HTTP/1.1" 200 -


陽明山
U091e03d020e37cce02ba9fd2f03377ec
1763196670470
346478f22ee5472bb8eed462df48a855


127.0.0.1 - - [15/Nov/2025 16:51:13] "POST /callback HTTP/1.1" 200 -


In [2]:
import time

t = 1762579212668
time.localtime(t/1000)

time.struct_time(tm_year=2025, tm_mon=11, tm_mday=8, tm_hour=13, tm_min=20, tm_sec=12, tm_wday=5, tm_yday=312, tm_isdst=0)

In [ ]:
!pip install pillow

In [22]:
from PIL import Image

image = Image.open('test.jpg')
image.size

(960, 1706)

In [23]:
image.height

1706

In [26]:
image.width

960

In [ ]:
960 x 1706

In [11]:
import time

int(time.time())+60

1763185395

In [ ]:
!pip install google-cloud-storage

In [5]:
from google.cloud import storage

storage_client = storage.Client.from_service_account_json('service.json')

In [6]:
bucket = storage_client.bucket('myfirstprojectnumberonebucket')

In [12]:
blob = bucket.blob('bbb.jpg')
blob.upload_from_filename('donav.jpg')

In [13]:
blob.generate_signed_url(int(time.time())+60)

'https://storage.googleapis.com/myfirstprojectnumberonebucket/bbb.jpg?Expires=1763185567&GoogleAccessId=myfirstserviceaccount%40firm-braid-476903-s2.iam.gserviceaccount.com&Signature=oVP7UhKydbTG8PktG%2FXmH1FOsd8%2F2g50PMaiCRTqP8kxlCwKvg2maYRPZvuE4%2BE7xi5A0kr3pBvQd8zMlQINVymLmubAwq9MG1AvOa2E8YkR3j8C%2F0TrI%2FlbSDUJAXpaS97kP7FQSqNTQvlqVJGjZfBI%2B78xEZea7YJ%2Fx5VxsSyx0Ehum2LW%2B15MwUkFFT4%2Bc%2BRNtwYcquv%2BtQ%2Fjv3VX56nCwQCIeovhTeoVO5cEreMin3HJ5alVgOfDnohZ3tGWwUCGLFpFPmM23L7jKpMvGT1TawVcxM2fNUJqUURXMU0mTV%2FsVLJ8Dp0FDrYPBrt2nkrx2Rz9YcDWPjlAUdQlAQ%3D%3D'

In [35]:
bucket.blob('abc.jpg').upload_from_filename('donav.jpg')

In [ ]:
help(blob.upload_from_string)

In [20]:
with open('me.json', encoding='utf-8') as f:
    me = json.load(f)

In [27]:
me

{'type': 'bubble',
 'header': {'type': 'box',
  'layout': 'baseline',
  'contents': [{'type': 'text',
    'text': '粉絲一覽',
    'size': '4xl',
    'align': 'center',
    'color': '#FF0000'}]},
 'hero': {'type': 'image',
  'url': 'https://cdn.britannica.com/94/256194-050-DD861124/Shohei-Ohtani-Los-Angeles-Dodgers.jpg',
  'size': 'full',
  'aspectRatio': '4:3',
  'action': {'type': 'uri', 'uri': 'https://line.me/'}},
 'body': {'type': 'box',
  'layout': 'vertical',
  'contents': [{'type': 'box',
    'layout': 'vertical',
    'margin': 'lg',
    'spacing': 'sm',
    'contents': [{'type': 'box',
      'layout': 'baseline',
      'spacing': 'sm',
      'contents': [{'type': 'text',
        'text': 'User ID',
        'color': '#0000FF',
        'size': 'sm',
        'flex': 0},
       {'type': 'text',
        'text': 'U12345678901234567890123456789012',
        'wrap': False,
        'color': '#666666',
        'size': 'sm',
        'flex': 5}]},
     {'type': 'box',
      'layout': 'baseline'

In [25]:
car = {"type": "carousel", "contents": [me, me]}

In [26]:
car

{'type': 'carousel',
 'contents': [{'type': 'bubble',
   'header': {'type': 'box',
    'layout': 'baseline',
    'contents': [{'type': 'text',
      'text': '粉絲一覽',
      'size': '4xl',
      'align': 'center',
      'color': '#FF0000'}]},
   'hero': {'type': 'image',
    'url': 'https://cdn.britannica.com/94/256194-050-DD861124/Shohei-Ohtani-Los-Angeles-Dodgers.jpg',
    'size': 'full',
    'aspectRatio': '4:3',
    'action': {'type': 'uri', 'uri': 'https://line.me/'}},
   'body': {'type': 'box',
    'layout': 'vertical',
    'contents': [{'type': 'box',
      'layout': 'vertical',
      'margin': 'lg',
      'spacing': 'sm',
      'contents': [{'type': 'box',
        'layout': 'baseline',
        'spacing': 'sm',
        'contents': [{'type': 'text',
          'text': 'User ID',
          'color': '#0000FF',
          'size': 'sm',
          'flex': 0},
         {'type': 'text',
          'text': 'U12345678901234567890123456789012',
          'wrap': False,
          'color': '#66666

In [21]:
me

{'type': 'bubble',
 'header': {'type': 'box',
  'layout': 'baseline',
  'contents': [{'type': 'text',
    'text': '粉絲一覽',
    'size': '4xl',
    'align': 'center',
    'color': '#FF0000'}]},
 'hero': {'type': 'image',
  'url': 'https://cdn.britannica.com/94/256194-050-DD861124/Shohei-Ohtani-Los-Angeles-Dodgers.jpg',
  'size': 'full',
  'aspectRatio': '4:3',
  'action': {'type': 'uri', 'uri': 'https://line.me/'}},
 'body': {'type': 'box',
  'layout': 'vertical',
  'contents': [{'type': 'box',
    'layout': 'vertical',
    'margin': 'lg',
    'spacing': 'sm',
    'contents': [{'type': 'box',
      'layout': 'baseline',
      'spacing': 'sm',
      'contents': [{'type': 'text',
        'text': 'User ID',
        'color': '#0000FF',
        'size': 'sm',
        'flex': 0},
       {'type': 'text',
        'text': 'U12345678901234567890123456789012',
        'wrap': False,
        'color': '#666666',
        'size': 'sm',
        'flex': 5}]},
     {'type': 'box',
      'layout': 'baseline'